In [2]:
from zipfile import ZipFile
import os
from tensorflow import keras
import pandas as pd


In [3]:
uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")
zip_file = ZipFile(zip_path)
zip_file.extractall()
csv_path = "jena_climate_2009_2016.csv"

df = pd.read_csv(csv_path)


13568290/13568290 [==============================] - 1s 0us/step


In [6]:
df.shape

(420551, 15)